##Для этого ДЗ хочу использовать возможность сдать после мягкого дедлайна без потери баллов


#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [ ]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning --quiet

     |████████████████████████████████| 800 kB 4.0 MB/s 
     |████████████████████████████████| 125 kB 53.4 MB/s 
     |████████████████████████████████| 512 kB 47.7 MB/s 


Скачаем данные

In [ ]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec 18 13:05 __MACOSX
drwxr-xr-x 1 root root 4096 Dec 16 00:01 sample_data


Посмотрим как выглядит один из файлов разметки

In [ ]:
!cat data/train/apple_3.xml

<annotation>
	<folder>train</folder>
	<filename>apple_3.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1000</width>
		<height>708</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>584</xmin>
			<ymin>438</ymin>
			<xmax>867</xmax>
			<ymax>708</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>492</xmin>
			<ymin>141</ymin>
			<xmax>740</xmax>
			<ymax>394</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>176</xmin>
			<ymin>199</ymin>
			<xmax>490</xmax>
			<ymax>466</ymax>
		</bndbox>
	</object>
	<obj

## Релизуйте выборку для YoloV1 - 2 балла

In [ ]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict
import shutil


from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.base import defaultdict

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch
from albumentations.pytorch import ToTensorV2

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
import sklearn.metrics
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#код взят из https://youtu.be/BVDRoDbVIyE
def divide_by_extention(path):
  extensions = {
            "jpg": "image",
            "xml": "boxes"
  }
  for extension, folder_name in extensions.items():
      files = glob.glob(os.path.join(path, f"*.{extension}"))
      print(f"[*] Найдено {len(files)} файла(-ов) с расширением {extension}.")
      if not os.path.isdir(os.path.join(path, folder_name)) and files:
          os.mkdir(os.path.join(path, folder_name))
          print(f"[+] Создана папка {folder_name}.")

      for file in files:
          basename = os.path.basename(file)
          dst = os.path.join(path, folder_name, basename)
          print(f"[*] Перенесён файл '{file}' в {dst}")
          shutil.move(file, dst)

In [ ]:
divide_by_extention("/content/data/test")
divide_by_extention("/content/data/train")

[*] Найдено 60 файла(-ов) с расширением jpg.
[+] Создана папка image.
[*] Перенесён файл '/content/data/test/apple_84.jpg' в /content/data/test/image/apple_84.jpg
[*] Перенесён файл '/content/data/test/orange_85.jpg' в /content/data/test/image/orange_85.jpg
[*] Перенесён файл '/content/data/test/apple_93.jpg' в /content/data/test/image/apple_93.jpg
[*] Перенесён файл '/content/data/test/mixed_21.jpg' в /content/data/test/image/mixed_21.jpg
[*] Перенесён файл '/content/data/test/apple_82.jpg' в /content/data/test/image/apple_82.jpg
[*] Перенесён файл '/content/data/test/orange_92.jpg' в /content/data/test/image/orange_92.jpg
[*] Перенесён файл '/content/data/test/apple_86.jpg' в /content/data/test/image/apple_86.jpg
[*] Перенесён файл '/content/data/test/apple_79.jpg' в /content/data/test/image/apple_79.jpg
[*] Перенесён файл '/content/data/test/banana_94.jpg' в /content/data/test/image/banana_94.jpg
[*] Перенесён файл '/content/data/test/orange_81.jpg' в /content/data/test/image/orange

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [ ]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, image_dir, boxes_dir, transforms=None):
        self.image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
        self.box_paths = glob.glob(os.path.join(boxes_dir,"*.xml"))

        assert len(self.image_paths) == len(self.box_paths)

        self.transforms = transforms

    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx])

        for i in range(len(boxes)):
          boxes[i] = self.__convert_to_yolo_box_params(box_coordinates=boxes[i], im_h = image.shape[0], im_w = image.shape[1])

        target_tensor = defaultdict()

        target_tensor["boxes"] = torch.tensor(boxes, dtype = torch.float32)
        target_tensor["class_labels"] = torch.tensor(class_labels, dtype = torch.float32)

        if self.transforms:
          transformed = self.transforms(image = image, bboxes = boxes, class_labels = class_labels)
          image = transformed["image"]
          target_tensor["boxes"] = transformed["bboxes"]
          target_tensor["class_labels"]  = transformed["class_labels"]
          

          
        return image, target_tensor

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      print

      with open(xml_filename) as fd:
        doc = xmltodict.parse(fd.read())
        if type(doc["annotation"]["object"]) is list:
          for i in doc["annotation"]["object"]:
            boxes.append(list(map(int,(i["bndbox"].values()))))
            class_labels.append(class2tag[i["name"]])
        else: 
          class_labels=(class2tag[doc["annotation"]["object"]["name"]])
          boxes=(list(map(int,(doc["annotation"]["object"]["bndbox"].values()))))
      return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = []

      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
      return ans

In [ ]:
WIDTH, HEIGHT = 448, 448

train_transform = A.Compose([A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.HorizontalFlip(p=0.5),
        A.Resize(height=HEIGHT, width=WIDTH),
        ToTensorV2(),
        ],bbox_params=A.BboxParams(format='yolo',label_fields=['class_labels']))
test_transform = A.Compose([A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.HorizontalFlip(p=0.5),
        A.Resize(height=HEIGHT, width=WIDTH),
        ToTensorV2(),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [ ]:
train_dataset = FruitDataset(
    transforms=train_transform,
    image_dir='/content/data/train/image',
    boxes_dir ='/content/data/train/boxes',
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    image_dir='/content/data/test/image',
    boxes_dir='/content/data/test/boxes',
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
print("Тесты успешно пройдены")


Тесты успешно пройдены


In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4, 
    shuffle=False
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [ ]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробок
 и один класс

In [ ]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C
        self.output_shape = self.S * self.S * (5 * self.B + self.C)
 
        self.block1 = CNNBlock(in_channels = 3, out_channels = 64, stride = 2, kernel_size = 7, is_max_pool = False)
        self.block2 = CNNBlock(in_channels = 64, out_channels = 128, dilation = 4, kernel_size = 3, is_max_pool = True)
        self.block3 = CNNBlock(in_channels = 128, out_channels=256, stride = 2, kernel_size = 3, is_max_pool = False)
        self.block4 = CNNBlock(in_channels = 256, out_channels=512, dilation = 4, kernel_size = 3, is_max_pool = True)
        self.block5 = CNNBlock(in_channels = 512, out_channels=1024, stride = 2, kernel_size = 3, is_max_pool = True)
        self.block6 = CNNBlock(in_channels = 1024, out_channels=1024, kernel_size = 1, is_max_pool = True)
        self.block7 = CNNBlock(in_channels = 1024, out_channels=4096, kernel_size = 1, is_max_pool = True)

        self.fc = nn.Linear(in_features = 4096, out_features = self.output_shape)
 
        


    def forward(self, x):

      x = self.block1.forward(x=x)
      x = self.block2.forward(x=x)
      x = self.block3.forward(x=x)
      x = self.block4.forward(x=x)
      x = self.block5.forward(x=x)
      x = self.block6.forward(x=x)
      x = self.block7.forward(x=x)
      x = self.fc(x.reshape(-1))

      return x




# Убедитесь на одном изображении, что предсказания вашей модели имеют верное количество значений

temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

testing_image = train_dataset[0][0]

assert temp_model(testing_image.unsqueeze(0)).reshape(-1).shape[0] == expected_output_shape

## Реализуйте YoloLoss - 3 балла

In [ ]:
#https://blog.flaport.net/yolo-part-1.html
def iou_wh(bboxes1, bboxes2):
    """ calculate iou between each bbox in `bboxes1` with each bbox in `bboxes2`
    
    The bboxes should be defined by their width and height and are centered around (0,0)
    
    """ 
    
    w1 = bboxes1[..., 0].view(-1)
    h1 = bboxes1[..., 1].view(-1)
    w2 = bboxes2[..., 0].view(-1)
    h2 = bboxes2[..., 1].view(-1)

    intersections = torch.min(w1[:,None],w2[None,:]) * torch.min(h1[:,None],h2[None,:])
    unions = (w1 * h1)[:,None] + (w2 * h2)[None,:] - intersections
    ious = (intersections / unions).reshape(*bboxes1.shape[:-1], *bboxes2.shape[:-1])

    return ious

In [ ]:
#https://blog.flaport.net/yolo-part-2.html
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.cel = torch.nn.CrossEntropyLoss(reduction='sum')

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
      nT = target.shape[1]
      nA = self.anchors.shape[0]
      nB, _, nH, nW = predictions.shape
      nPixels = nH * nW
      nAnchors = nA * nPixels
      target = target.to(dtype=predictions.dtype)
      predictions = predictions.view(nB, nA, -1, nH, nW).permute(0, 1, 3, 4, 2)
      nC = predictions.shape[-1] - 5
      self.seen += nB

      anchors = self.anchors.to(dtype=predictions.dtype)
      coord_mask = torch.zeros(nB, nA, nH, nW, 1, requires_grad=False, dtype=predictions.dtype)
      conf_mask = torch.ones(nB, nA, nH, nW, requires_grad=False, dtype=predictions.dtype) * self.lambda_noobj
      cls_mask = torch.zeros(nB, nA, nH, nW, requires_grad=False, dtype=torch.bool)
      tcoord = torch.zeros(nB, nA, nH, nW, 4, requires_grad=False, dtype=predictions.dtype)
      tconf = torch.zeros(nB, nA, nH, nW, requires_grad=False, dtype=predictions.dtype)
      tcls = torch.zeros(nB, nA, nH, nW, requires_grad=False, dtype=predictions.dtype)

      coord = torch.cat([
            predictions[:, :, :, :, 0:1].sigmoid(),  
            predictions[:, :, :, :, 1:2].sigmoid(),  
            predictions[:, :, :, :, 2:3],  
            predictions[:, :, :, :, 3:4],  
        ], -1)
      
      range_target, range_predictions = torch.meshgrid(
            torch.arange(nH, dtype=predictions.dtype),
            torch.arange(nW, dtype=predictions.dtype),
        )
      anchor_predictions, anchor_target = anchors[:, 0], anchors[:, 1]

      x = torch.cat([
            (x[:, :, :, :, 0:1].sigmoid() + range_predictions[None,None,:,:,None]),  
            (x[:, :, :, :, 1:2].sigmoid() + range_target[None,None,:,:,None]), 
            (x[:, :, :, :, 2:3].exp() * anchor_predictions[None,:,None,None,None]),  
            (x[:, :, :, :, 3:4].exp() * anchor_target[None,:,None,None,None]),  
            x[:, :, :, :, 4:5].sigmoid(), 
            x[:, :, :, :, 5:], 
        ], -1)

      conf = predictions[..., 4]
      cls = predictions[..., 5:].reshape(-1, nC)
      predictions = predictions[..., :4].detach() 

      if self.seen < self.coord_prefill:
            coord_mask.fill_(np.sqrt(.01 / self.lambda_coord))
            tcoord[..., 0].fill_(0.5)
            tcoord[..., 1].fill_(0.5)

      for b in range(nB):
            gt = target[b][(target[b, :, -1] >= 0)[:, None].expand_as(target[b])].view(-1, 6)[:,:4]
            gt[:, ::2] *= nW
            gt[:, 1::2] *= nH
            if gt.numel() == 0:  
                continue

            
            iou_gt_pred = intersection_over_union(gt, x[b:(b+1)].view(-1, 4))
            mask = (iou_gt_pred > self.threshold).sum(0) >= 1
            conf_mask[b][mask.view_as(conf_mask[b])] = 0

            
            iou_gt_anchors = iou_wh(gt[:,2:], anchors)
            _, best_anchors = iou_gt_anchors.max(1)

           
            nGT = gt.shape[0]
            gi = gt[:, 0].clamp(0, nW-1).long()
            gj = gt[:, 1].clamp(0, nH-1).long()

            conf_mask[b, best_anchors, gj, gi] = self.lambda_obj
            tconf[b, best_anchors, gj, gi] = iou_gt_pred.view(nGT, nA, nH, nW)[torch.arange(nGT), best_anchors, gj, gi]
            coord_mask[b, best_anchors, gj, gi, :] = (2 - (gt[:, 2] * gt[:, 3]) / nPixels)[..., None]
            tcoord[b, best_anchors, gj, gi, 0] = gt[:, 0] - gi.float()
            tcoord[b, best_anchors, gj, gi, 1] = gt[:, 1] - gj.float()
            tcoord[b, best_anchors, gj, gi, 2] = (gt[:, 2] / anchors[best_anchors, 0]).log()
            tcoord[b, best_anchors, gj, gi, 3] = (gt[:, 3] / anchors[best_anchors, 1]).log()
            cls_mask[b, best_anchors, gj, gi] = 1
            tcls[b, best_anchors, gj, gi] = target[b, torch.arange(nGT), -1]

      coord_mask = coord_mask.sqrt()
      conf_mask = conf_mask.sqrt()
      tcls = tcls[cls_mask].view(-1).long()
      cls_mask = cls_mask.view(-1, 1).expand(nB*nA*nH*nW, nC)
      cls = cls[cls_mask].view(-1, nC)

      loss_coord = self.lambda_coord * self.mse(coord*coord_mask, tcoord*coord_mask) / (2 * nB)
      loss_conf = self.mse(conf*conf_mask, tconf*conf_mask) / (2 * nB)
      loss_cls = self.lambda_cls * self.cel(cls, tcls) / nB
      return loss_coord + loss_conf + loss_cls
    

        
        

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
#https://habr.com/ru/company/jetinfosystems/blog/498294/
def non_max_suppression(bboxes, iou_threshold, threshold):
  sorted_boxes = sorted(bboxes, key = abs(bboxes - threshold))
  res = []
  for i in sorted_boxes:
    res.append(i)
    new_sorted = sorted_boxes / i
    for j in new_sorted:
      if intersection_over_union(i, j) > iou_threshold:
        sorted_boxes.remove(j)
    return res

#https://blog.paperspace.com/mean-average-precision/
def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
  precision = []
  recall = []
  y_pred = []
  for i in range(len(pred_boxes)):
    if intersection_over_union(pred_boxes[i], true_boxes[i]) > iou_threshold:
      y_pred.append("positive")
    else:
      y_pred.append("negative")
  precision = sklearn.metrics.precision_score(y_true = true_boxes, y_pred = y_pred, pos_labels = "positive")
  recall = sklearn.metrics.recall_score(y_true = true_boxes, y_pred = y_pred, pos_labels = "positive")
  av_prec = np.sum((recall[:-1] - recall[1:]) * precision[:-1])
  return 1/len(pred_boxes) * av_prec


def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    all_pred_boxes = model(loader)
    all_true_boxes = non_max_suppression(all_pred_boxes, iou_threshold, threshold)
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model = # YOUR CODE
n_epochs = # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image